In [ ]:
#code comes from simpletransformers in https://github.com/bit-ml/date/blob/master/simpletransformers/simpletransformers/language_modeling/language_modeling_model.py
from language_modeling import LanguageModelingModel
import logging
masks_ = pkl.load(open('./mask.pkl', 'rb'))

print('Using subset ', subset)

if subset == 1:
    subset = ['business']
elif subset == 2:
    subset = ['sci']
elif subset == 3:
    subset = ['sports']
elif subset == 4:
    subset = ['world']

def run_exps():
    logging.basicConfig(level=logging.INFO)
    transformers_logger = logging.getLogger("transformers")
    transformers_logger.setLevel(logging.WARNING)

    train_args = {
        #"fp16": False,
        "use_multiprocessing": False,
        "reprocess_input_data": False,
        "overwrite_output_dir": True,
        "num_train_epochs": 20,
        "save_eval_checkpoints": False,
        "save_model_every_epoch": False,
        "vocab_size": 52000,
        "tokenizer_name": "bert-base-uncased",
        "generator_config": {
            "embedding_size": 128,
            "hidden_size": 16,
            "num_hidden_layers": 1,
        },
        "discriminator_config": {
            "hidden_dropout_prob": 0.5,
            "attention_probs_dropout_prob": 0.5,
            "embedding_size": 128,
            "hidden_size": 16,
            "num_hidden_layers": 4,
        },
        "mlm_lr_ratio": 1,
    }

    for subset_r in tqdm(subset):
        print('-' * 10, '\n', f'SUBSET: {subset_r}', '-' * 10)


        train_file = f"./ag_od/train/{subset_r}.txt"
        test_file = f"./ag_od/test/{subset_r}.txt"
        outlier_file = f"./ag_od/test/{subset_r}-outliers.txt"

        model = LanguageModelingModel("electra",
                                      None,
                                      masks=masks_,
                                      args=train_args,
                                      train_files=train_file,
                                      use_cuda=True)

        model.train_model_anomaly(train_file,
                                  eval_file=test_file,
                                  eval_file_outlier=outlier_file,
                                  sched_params=sched_params)

